In [177]:
!pip install konlpy
!pip install tensorflow
!pip install -q -U tensorflow-text
!pip install -q tf-models-official
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [178]:
import csv
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
from konlpy.tag import Okt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [179]:
data_atmosphere = pd.read_csv('restaurant_atmosphere.csv')
data_cost = pd.read_csv('restaurant_cost.csv')
#data_service = pd.read_csv('restaurant_service2.csv',on_bad_lines='skip')
data_taste = pd.read_csv('restaurant_taste.csv')
data_visit = pd.read_csv('restaurant_visit.csv')
print(data_taste)

                                                context  key
0     맛보다 수 있다 맛 좋다 요 정말 맛있다 고소하다 맛 특징 라떼 고소하다 부드럽다 ...    1
1     맛있다 하다 라떼 맛있다 들다 니다 케이크 맛있다 라떼 맛있다 고소하다 고르다 좋다...    1
2     맛있다 픽 버터 다 맛있다 고소하다 맛 당근 고소하다 배곧 스콘 듯 고소하다 텁텁 ...    1
3                  맛있다 커피 디저트 들다 맛있다 고소하다 특징 이다 라떼 고소하다    1
4     맛있다 티라미슈 좋아하다 맛있다 디저트 카페 촉촉하다 맛있다 후 맛있다 촉촉하다 맛...    1
...                                                 ...  ...
9635                                최고이다  너무 맛 크다플 최고이다    1
9636                                          맛있다 자다 먹다    1
9637                              맛있다  취향 차이다 아메리카노 맛있다    1
9638                                   맛있다  배곧 커피 다 맛있다    1
9639                                             커피 맛있다    1

[9640 rows x 2 columns]


In [180]:
train_taste_data = data_taste.sample(frac=0.8, random_state=25)
test_taste_data = data_taste.drop(train_taste_data.index)
train_cost_data = data_cost.sample(frac=0.8, random_state=25)
test_cost_data = data_cost.drop(train_cost_data.index)
train_visit_data = data_visit.sample(frac=0.8, random_state=25)
test_visit_data = data_visit.drop(train_visit_data.index)
train_atmosphere_data = data_atmosphere.sample(frac=0.8, random_state=25)
test_atmosphere_data = data_atmosphere.drop(train_atmosphere_data.index)

In [181]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.python.keras.layers import Embedding, Dense, LSTM
from keras.models import Sequential
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(train_taste_data['context'])
tokenizer.fit_on_texts(train_cost_data['context'])
tokenizer.fit_on_texts(train_visit_data['context'])
tokenizer.fit_on_texts(train_atmosphere_data['context'])

train_taste_x = tokenizer.texts_to_sequences(train_taste_data['context'])
train_taste_x = pad_sequences(train_taste_x, padding='post',maxlen = 50)
train_taste_y = np.array(train_taste_data['key'])

train_cost_x = tokenizer.texts_to_sequences(train_cost_data['context'])
train_cost_x = pad_sequences(train_cost_x, padding='post',maxlen = 50)
train_cost_y = np.array(train_cost_data['key'])

train_visit_x = tokenizer.texts_to_sequences(train_visit_data['context'])
train_visit_x = pad_sequences(train_visit_x, padding='post',maxlen = 50)
train_visit_y = np.array(train_visit_data['key'])

train_atmosphere_x = tokenizer.texts_to_sequences(train_atmosphere_data['context'])
train_atmosphere_x = pad_sequences(train_atmosphere_x, padding='post',maxlen = 50)
train_atmosphere_y = np.array(train_atmosphere_data['key'])

print(train_taste_x[:5])
print(train_cost_x[:5])
print(train_visit_x[:5])
print(train_atmosphere_x[:5])

[[   1  162 2528    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0]
 [  76  214  144   41   34   10    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0]
 [  59   10    1   10    1    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0]
 [   1  607 2529    7   10   83   51    1  163    1 2530    1    6  608
   762 2531    6    3  763 1304    3   89  148   12    3   16    3   16
     3   89  608   16    0    0    0    0    0    0    0    0    0    0
     0 

In [182]:
model = tf.keras.Sequential([    
  tf.keras.layers.Embedding(20000, 100, input_length=50),    
  tf.keras.layers.LSTM(128),    
  tf.keras.layers.Dense(1, activation='sigmoid')])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_taste_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=False)


In [183]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
history = model.fit(train_taste_x, train_taste_y, epochs=10, callbacks=[es, mc], batch_size=64, validation_split=0.2)

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_22 (Embedding)    (None, 50, 100)           2000000   
                                                                 
 lstm_22 (LSTM)              (None, 128)               117248    
                                                                 
 dense_22 (Dense)            (None, 1)                 129       
                                                                 
Total params: 2,117,377
Trainable params: 2,117,377
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
97/97 [==============================] - ETA: 0s - loss: 0.1863 - accuracy: 0.9596
Epoch 1: saving model to best_taste_model.h5
97/97 [==============================] - 35s 310ms/step - loss: 0.1863 - accuracy: 0.9596 - val_loss: 0.1848 - val_accuracy: 0.9540
Epoch 2/10
97/97 [==================

In [184]:
test_x = tokenizer.texts_to_sequences(test_taste_data['context'])
test_x = pad_sequences(test_x, padding='post',maxlen =50)
test_y = np.array(test_taste_data['key'])

model.evaluate(test_x,test_y)

61/61 [==============================] - 2s 30ms/step - loss: 0.0470 - accuracy: 0.9927


[0.04700787737965584, 0.9927386045455933]

In [185]:
model2 = tf.keras.Sequential([    
  tf.keras.layers.Embedding(20000, 100, input_length=50),    
  tf.keras.layers.LSTM(128),    
  tf.keras.layers.Dense(1, activation='sigmoid')])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_cost_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=False)

In [186]:
model2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model2.summary()
history = model2.fit(train_cost_x, train_cost_y, epochs=10, callbacks=[es, mc], batch_size=64, validation_split=0.2)

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_23 (Embedding)    (None, 50, 100)           2000000   
                                                                 
 lstm_23 (LSTM)              (None, 128)               117248    
                                                                 
 dense_23 (Dense)            (None, 1)                 129       
                                                                 
Total params: 2,117,377
Trainable params: 2,117,377
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
9/9 [==============================] - ETA: 0s - loss: 0.6371 - accuracy: 0.6991
Epoch 1: saving model to best_cost_model.h5
9/9 [==============================] - 6s 305ms/step - loss: 0.6371 - accuracy: 0.6991 - val_loss: 0.6474 - val_accuracy: 0.6620
Epoch 2/10
9/9 [==========================

In [187]:
test_x = tokenizer.texts_to_sequences(test_cost_data['context'])
test_x = pad_sequences(test_x, padding='post',maxlen =50)
test_y = np.array(test_cost_data['key'])

model2.evaluate(test_x,test_y)

6/6 [==============================] - 0s 28ms/step - loss: 0.6406 - accuracy: 0.6610


[0.6405514478683472, 0.6610169410705566]

In [188]:
model3 = tf.keras.Sequential([    
  tf.keras.layers.Embedding(20000, 100, input_length=50),    
  tf.keras.layers.LSTM(128),    
  tf.keras.layers.Dense(1, activation='sigmoid')])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_visit_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=False)

In [189]:
model3.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model3.summary()
history = model3.fit(train_visit_x, train_visit_y, epochs=10, callbacks=[es, mc], batch_size=64, validation_split=0.2)

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_24 (Embedding)    (None, 50, 100)           2000000   
                                                                 
 lstm_24 (LSTM)              (None, 128)               117248    
                                                                 
 dense_24 (Dense)            (None, 1)                 129       
                                                                 
Total params: 2,117,377
Trainable params: 2,117,377
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
3/3 [==============================] - ETA: 0s - loss: 0.6200 - accuracy: 0.8228
Epoch 1: saving model to best_visit_model.h5
3/3 [==============================] - 5s 640ms/step - loss: 0.6200 - accuracy: 0.8228 - val_loss: 1.3552 - val_accuracy: 0.6750
Epoch 2/10
3/3 [=========================

In [190]:
test_x = tokenizer.texts_to_sequences(test_visit_data['context'])
test_x = pad_sequences(test_x, padding='post',maxlen =50)
test_y = np.array(test_visit_data['key'])

model3.evaluate(test_x,test_y)

2/2 [==============================] - 0s 29ms/step - loss: 0.4065 - accuracy: 0.8980


[0.4064987301826477, 0.8979591727256775]

In [191]:
model4 = tf.keras.Sequential([    
  tf.keras.layers.Embedding(20000, 100, input_length=50),    
  tf.keras.layers.LSTM(128),    
  tf.keras.layers.Dense(1, activation='sigmoid')])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_atmosphere_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=False)

In [192]:
model4.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model4.summary()
history = model4.fit(train_atmosphere_x, train_atmosphere_y, epochs=10, callbacks=[es, mc], batch_size=64, validation_split=0.2)

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_25 (Embedding)    (None, 50, 100)           2000000   
                                                                 
 lstm_25 (LSTM)              (None, 128)               117248    
                                                                 
 dense_25 (Dense)            (None, 1)                 129       
                                                                 
Total params: 2,117,377
Trainable params: 2,117,377
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
13/13 [==============================] - ETA: 0s - loss: 0.3081 - accuracy: 0.9450
Epoch 1: saving model to best_atmosphere_model.h5
13/13 [==============================] - 8s 347ms/step - loss: 0.3081 - accuracy: 0.9450 - val_loss: 0.3087 - val_accuracy: 0.9073
Epoch 2/10
13/13 [==============

In [193]:
test_x = tokenizer.texts_to_sequences(test_atmosphere_data['context'])
test_x = pad_sequences(test_x, padding='post',maxlen =50)
test_y = np.array(test_atmosphere_data['key'])

model4.evaluate(test_x,test_y)

8/8 [==============================] - 1s 29ms/step - loss: 0.2231 - accuracy: 0.9414


[0.22311000525951385, 0.94140625]